# Advanced Queries using Python and SQL

## Load in the Database 

In [1]:
from astrodb_utils import load_astrodb
from simple import REFERENCE_TABLES

SCHEMA_PATH = "../../simple/schema.yaml" 
recreatedb = False #Set to True or False
db = load_astrodb(
     "../../SIMPLE.sqlite", 
     recreatedb=recreatedb,  
     reference_tables=REFERENCE_TABLES, 
     felis_schema=SCHEMA_PATH
     )

## Let first use astrodb_utils.sql_query to run a SQL query directly 
AstrodbUtils's function sql_query acts as a wrapped for direct SQL queries, taking in two parameters: query (str), and fmt (str: pandas, astropy, defualt). 

In [32]:
from sqlalchemy import and_, or_

#Uncomment the following lines to see how to use AND and OR in queries

# Query with multiple conditions
print("\nQuery with multiple conditions, filtering sources with dec > 0 and ra < 200 degrees:")
print(f"Showing the first 10 results out of {len(db.query(db.Sources).filter(db.Sources.c.dec > 0).filter(db.Sources.c.ra < 200).all())} sources:")
db.query(db.Sources).filter(db.Sources.c.dec > 0).filter(db.Sources.c.ra < 200).limit(10).table()

# #  Query with AND
# print("\nQuery with AND, filtering sources with dec > 0 and ra > 200 degrees:")
# print(f"Showing the first 10 results out of {len(db.query(db.Sources).filter(and_(db.Sources.c.dec > 0, db.Sources.c.ra > 200)).all())} sources:")
# db.query(db.Sources).filter(and_(db.Sources.c.dec > 0, db.Sources.c.ra > 200)).limit(10).table()


# #  Query with OR
# print("\nQuery with OR, filtering sources with dec < 0 or ra > 200 degrees:")
# print(f"Showing the first 10 results out of {len(db.query(db.Sources).filter(or_(db.Sources.c.dec < 0, db.Sources.c.ra > 200)).all())} sources:")
# db.query(db.Sources).filter(or_(db.Sources.c.dec < 0, db.Sources.c.ra > 200)).limit(10).table()



Query with multiple conditions, filtering sources with dec > 0 and ra < 200 degrees:
Showing the first 10 results out of 1334 sources:


source,ra,dec,epoch,equinox,shortname,reference,other_references,comments
str24,float64,float64,object,object,object,str11,object,object
2MASS J03552014+1439297,58.83375,14.658056,None,None,None,Cruz07,None,None
PSO J024.4369+09.1360,24.4372,9.1366,None,2000,None,Skrz16,None,Best20 lists discovery as Skrz16; Best20a
SDSS J094903.15+264944.2,147.2633,26.8288,None,2000,None,Schm10.1808,None,None
2MASS J09373487+2931409,144.395292,29.528028,None,None,0937+2931,Burg02.421,None,None
HIP 6407B,20.5707,3.5232,None,2000,None,Deac14.119,None,None
2MASS J11260310+4819256,171.5128,48.3239,None,2000,None,Kell15,None,None
LHS 1937,115.27833,17.645833,None,None,None,Luyt79.5,None,None
ULAS J095047.28+011734.3,147.6971,1.2925,None,2000,None,Burn13,None,None
2MASSI J1029216+162652,157.3404,16.4477,None,2000,None,Kirk00,None,None


### Query the count of sources in the database

In [33]:
# Query the total number of objects within SIMPLE
result = db.sql_query("SELECT COUNT(*) FROM Sources", fmt='astropy')
total_objects = result[0][0]


print(f"Total number of objects in the SIMPLE database: {total_objects}")

Total number of objects in the SIMPLE database: 3619


### Query Sources by String 

In [34]:
wise_sources_query = "SELECT * FROM Sources WHERE source LIKE '%WISE%'"
wise_sources = db.sql_query(wise_sources_query, fmt='astropy')

print(f"Found {len(wise_sources)} WISE sources in the database")
print("\nFirst Three WISE Sources:")
for source in wise_sources[:3]:  # Display first 3 sources
    print(source)

Found 616 WISE sources in the database

First Three WISE Sources:
          source                   ra               dec         epoch equinox shortname reference other_references comments
-------------------------- ----------------- ------------------ ----- ------- --------- --------- ---------------- --------
CWISEP J230158.30-645858.3 345.4929166666666 -64.98286111111112  None    None      None Meis20.74             None     None
         source             ra     dec    epoch equinox shortname reference other_references comments
------------------------ ------- -------- ----- ------- --------- --------- ---------------- --------
WISE J200403.17-263751.7 301.013 -26.6293  None    2000      None    Thom13             None     None
          source                  ra                 dec         epoch equinox shortname reference  other_references comments
------------------------- ------------------ ------------------- ----- ------- --------- ---------- ---------------- --------
WISE

### Query by Program ID ("PID")

In [37]:
spectra_pid_query = "SELECT * FROM Spectra WHERE reference LIKE 'PID%'"
print(f"Found {len(db.sql_query(spectra_pid_query))} spectra with PID references in the database")
db.sql_query(spectra_pid_query, fmt='astropy')


Found 55 spectra with PID references in the database


source,access_url,original_spectrum,local_spectrum,regime,telescope,instrument,mode,observation_date,comments,reference,other_references
str24,str67,object,str43,str3,str7,str3,str7,str26,object,str8,object
VB 10,https://s3.amazonaws.com/bdnyc/Gl+752B+%285-35um%29.txt,None,$BDNYC_spectra/IRS/Gl 752B (5-35um).txt,mir,Spitzer,IRS,Missing,2005-10-11 00:00:00.000000,None,PID29,None
2MASS J10042066+5022596,https://s3.amazonaws.com/bdnyc/IRS/G196-3B.txt,None,$BDNYC_spectra/IRS/G196-3B.txt,mir,Spitzer,IRS,Missing,2004-04-17 04:40:11.761000,None,PID51,None
2MASS J10224821+5825453,https://s3.amazonaws.com/bdnyc/IRS/1022%2B5825.txt,None,$BDNYC_spectra/IRS/1022+5825.txt,mir,Spitzer,IRS,Missing,2005-04-16 13:45:42.752000,None,PID3136,None
2MASS J12392727+5515371,https://s3.amazonaws.com/bdnyc/1239%2B5515.txt,None,$BDNYC_spectra/IRS/1239+5515.txt,mir,Spitzer,IRS,Missing,2004-04-18 02:01:06.605000,None,PID51,None
2MASS J00001354+2554180,https://s3.amazonaws.com/bdnyc/IRS/0000%2B2554%20%285-14.5um%29.txt,None,$BDNYC_spectra/IRS/0000+2554 (5-14.5um).txt,mir,Spitzer,IRS,Missing,2005-12-18 00:00:00.000000,None,PID20514,None
2MASS J07464256+2000321,https://s3.amazonaws.com/bdnyc/0746%2B2000.txt,None,$BDNYC_spectra/IRS/0746+2000.txt,mir,Spitzer,IRS,Missing,2004-03-27 00:32:11.624000,None,PID51,None
2MASS J20575409-0252302,https://s3.amazonaws.com/bdnyc/2057-0252.txt,None,$BDNYC_spectra/IRS/2057-0252.txt,mir,Spitzer,IRS,SL,2005-05-25 01:56:38.317000,None,PID3136,None
LP 440-52,https://s3.amazonaws.com/bdnyc/IRS/LHS377.txt,None,$BDNYC_spectra/IRS/LHS377.txt,mir,Spitzer,IRS,Missing,2005-07-01 00:00:00.000000,None,PID251,None
2MASS J12373919+6526148,https://s3.amazonaws.com/bdnyc/IRS/1237%2B6526.txt,None,$BDNYC_spectra/IRS/1237+6526.txt,mir,Spitzer,IRS,SL,2004-03-03 20:10:57.708000,None,PID51,None


## Using SQLAlchemy Query Functions

#### SQL via SQLalchemy 

In [16]:
from sqlalchemy import text

# Using connection from the existing database
with db.engine.connect() as conn:
    
    # Query all sources
    result = conn.execute(text("SELECT * FROM Sources LIMIT 10"))
    print("First 10 Sources (Raw SQL):")
    for row in result:
        print(f"Source: {row.source}, RA: {row.ra}, Dec: {row.dec}")

    # Find spectra with a specific instrument
    result = conn.execute(
        text("SELECT * FROM Spectra WHERE instrument = :instrument LIMIT 5"),
        {"instrument": "IRS"}
    )
    print("\nSpectra from IRS instrument (first 5):")
    for row in result:
        print(f"Source: {row.source}, Telescope: {row.telescope}")

        # Join query - find all spectra for sources with names containing "WISE"
        result = conn.execute(
            text("""
            SELECT Sources.source, Spectra.telescope, Spectra.instrument
            FROM Sources 
            JOIN Spectra ON Sources.source = Spectra.source
            WHERE Sources.source LIKE :pattern
            LIMIT 5
            """),
            {"pattern": "%WISE%"}
        )
        print("\nWISE sources with spectra (first 5):")
        for row in result:
            print(f"Source: {row.source}, Telescope: {row.telescope}, Instrument: {row.instrument}")
            
        # Aggregate query - count sources by publication reference
        result = conn.execute(
            text("""
            SELECT reference, COUNT(*) as count 
            FROM Sources 
            GROUP BY reference 
            ORDER BY count DESC
            LIMIT 3
            """)
        )
        print("\nTop 3 references by number of sources:")
        for row in result:
            print(f"Reference: {row.reference}, Count: {row.count}")


First 10 Sources (Raw SQL):
Source: 2MASS J03552014+1439297, RA: 58.83375, Dec: 14.658056
Source: 1RXS J115928.5-524717, RA: 179.864, Dec: -52.7886
Source: ULAS J114925.58-014343.2, RA: 177.3569, Dec: -1.7287
Source: DENIS-P J1756561-480509, RA: 269.2342, Dec: -48.086
Source: PSO J024.4369+09.1360, RA: 24.4372, Dec: 9.1366
Source: SDSS J161420.50+004643.6, RA: 243.585, Dec: 0.7785
Source: SDSS J094903.15+264944.2, RA: 147.2633, Dec: 26.8288
Source: SDSS J083717.21-000018.0, RA: 129.3215835, Dec: -0.004972
Source: 2MASS J06080232-2944590, RA: 92.0097, Dec: -29.7497
Source: 2MASS J09373487+2931409, RA: 144.395292, Dec: 29.528028

Spectra from IRS instrument (first 5):
Source: SDSS J083717.21-000018.0, Telescope: Spitzer

WISE sources with spectra (first 5):
Source: WISE J235402.77+024015.0, Telescope: JWST, Instrument: MIRI
Source: WISE J235402.77+024015.0, Telescope: JWST, Instrument: NIRSpec
Source: WISE J235402.77+024015.0, Telescope: HST, Instrument: WFC3
Source: WISEP J031325.96+780